In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## import libraries

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split,DataLoader,TensorDataset

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

## Reading files

In [ ]:
train=pd.read_csv("../input/digit-recognizer/train.csv")
test=pd.read_csv("../input/digit-recognizer/test.csv")

In [ ]:
test.head()

In [ ]:
train.head()

In [ ]:
len(train)

In [ ]:
len(test)

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
# Target variable
labels=train.label

In [ ]:
# Independent variables
train_ds=train.drop('label',axis=1)

In [ ]:
train_ds.head()

In [ ]:
plt.imshow(train_ds[1:2].values.reshape(28,28))
plt.axis("off")
print(labels[1])

## converting the data to numpy arrays

In [ ]:
train_ds=train_ds.values
test_ds=test.values
labels=labels.to_numpy()             #labels was actually a pandas series

## convert data to tensor

In [ ]:
train_ds=torch.tensor(train_ds)
test_ds =torch.tensor(test_ds)
labels = torch.tensor(labels)

## converting data into tensordataset

In [ ]:
traiin_ds=TensorDataset(train_ds,labels)

In [ ]:
traiin_ds[0:2]

## Define training and validation data

In [ ]:
train_ds , val_ds = random_split(traiin_ds,(32000,10000))

In [ ]:
train_loader=DataLoader(train_ds , batch_size=128, shuffle = True)
val_loader=DataLoader(val_ds , batch_size=128, shuffle = False)

In [ ]:
test_loader=DataLoader(test_ds , batch_size=128, shuffle = False)

In [ ]:
# for each image
input_size = 784
num_class = 10

## Cnn Model

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
                                                                        
        self.conv1 = nn.Conv2d(
                                in_channels=1,
                                out_channels=8,
                                kernel_size=(3, 3),
                                stride=(1, 1),
                                padding=(1, 1)
        )                                                                
        
        self.pool = nn.MaxPool2d(
                                 kernel_size=(2, 2),
                                 stride=(2, 2)
        )                                                               
                                                                        
        
        self.conv2 = nn.Conv2d(
                                in_channels=8,
                                out_channels=16,
                                kernel_size=(3, 3),
                                stride=(1, 1),
                                padding=(1, 1),
        )                                                            
        
        self.fc1 = nn.Linear(16 * 7 * 7, num_class)

    def forward(self, x):
        x = F.relu(self.conv1(x))                                      # 8,28,28
        x = self.pool(x)                                               # 8,14,14
        x = F.relu(self.conv2(x))                                      # 16,14,14
        x = self.pool(x)                                               #16,7,7
        
#         print(x.shape)
        x = x.reshape(x.shape[0], -1)
    
        x = self.fc1(x)
        return x
    
model = CNN().to('cuda')

In [ ]:
#optimizer
opt      = torch.optim.Adam
opt = opt(model.parameters())

## Train the model

In [ ]:
def fit(epochs,model,data):
    
    loss_fun = F.cross_entropy
    hist = []
    for Epoch in range(epochs):
        for img,label in data:
            img = img.reshape(img.shape[0],1,28,28)
            img=img.to('cuda')
            label= label.to('cuda')
#             print(img.shape)
            out = model(img/255)
            loss = loss_fun(out,label)
            loss.backward()
            opt.step()
            opt.zero_grad()
            hist.append(loss)
        
        if (Epoch+1)%50==0:
            print(f"Epoch:[{Epoch+1}/{epochs}] ; Loss : {loss:.4f}")
    return hist

In [ ]:
hist=fit(500,model,train_loader)

In [ ]:
def acc(data):
    accy=[]
    for img ,label in data:
        img = img.reshape(img.shape[0],1,28,28)
        img,label = img.to('cuda'),label.to('cuda')
        out=model(img/255)
#         out=F.softmax(out)
        _,pred_index=torch.max(out,dim=1)
        x=torch.sum(pred_index==label)/len(pred_index)
        x=x*100
        x=x.to("cpu").numpy()
        accy.append(x)
        
        
    return np.mean(accy)  

In [ ]:
acc(val_loader)

In [ ]:
submission= pd.read_csv("../input/digit-recognizer/sample_submission.csv")
submission.head()

In [ ]:
def prediction(data):
    LABEL=[]
    data = data.reshape(data.shape[0],1,28,28)
    data=data.to('cuda')
    out = model(data/255)
    out = F.softmax(out)     
    _,pred_index =torch.max(out,dim=1)
    LABEL.append(pred_index)   
    return LABEL

In [ ]:
test_ds.shape

In [ ]:
x=prediction(test_ds)
len(x[0])

In [ ]:
x

In [ ]:
x=x[0].to('cpu').numpy()

In [ ]:
submission["Label"]=x

In [ ]:
submission.to_csv("submission.csv",index=False)

## UPVOTE